# yolov5
version 2 

author: nmc-costa

info: testing version 2


# Setup
Go to tutorial_yolov5.ipnb for full setup (like cloning the yolov5 repo, etc...)

In [1]:
import os
import sys
import platform
import torch
from IPython.display import Image, clear_output  # to display images


clear_output()

#Define root dir dependent on OS
rdir='D:/external_datasets/' 
if str(platform.platform()).find('linux')>-1: rdir='/mnt/d/external_datasets/' 
print('OS: {}'.format(platform.platform()))
print('root dir: {}'.format(rdir))
print("Python: " + sys.version)

#cuda 
#os.environ["CUDA_VISIBLE_DEVICES"]="" #force CPU
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

OS: Windows-10-10.0.20251-SP0
root dir: D:/external_datasets/
Python: 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
Setup complete. Using torch 1.5.1 _CudaDeviceProperties(name='GeForce GTX 1070', major=6, minor=1, total_memory=8192MB, multi_processor_count=16)


## init vars

In [2]:
# DIRs [needed if you use %cd command multiple times ]
#script dir 
scriptdir=sys.path[0] #os.getcwd()
#yolov5 Dir
yolov5_dir=os.path.join(scriptdir, 'yolov5')
print(yolov5_dir)
#json2yolo Dir
json2yolo_dir=os.path.join(scriptdir, 'json2yolo')
print(json2yolo_dir)

C:\Users\nmc_costa\google_drive\projects\bosch_P19\research\python_ws\violent_action\yolo\yolov5
C:\Users\nmc_costa\google_drive\projects\bosch_P19\research\python_ws\violent_action\yolo\json2yolo


## 0. input data
#### Prepare the dataset for yolo format (CHECK OUT https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data ; https://medium.com/towards-artificial-intelligence/yolo-v5-is-here-custom-object-detection-tutorial-with-yolo-v5-12666ee1774e)

##### 1. Create labels and organize dataset in YOLO format (USE $JSON2YOLO$ folder)

Dataset Directory structure:
- dataset:
    - annotations:
        - samefoldername.json (WARNING: use 'train', 'val', 'test')
    - images:
        - samefoldername
    - labels:
        - samefoldername

Example :

dataset/images/train/000000109622.jpg  # image

dataset/labels/train/000000109622.txt  # label


In [27]:
source="mola"
datasets_root_dir='D:/external_datasets/'
json_dir='D:/external_datasets/MOLA/annotations/train_mola_mix_aggressive/'
outdir="D:/external_datasets/MOLA/yoloformat/mola128/"
img_number=128

In [ ]:
%cd $json2yolo_dir
!python json2yolo.py --source $source --datasets_root_dir $datasets_root_dir --json_dir $json_dir --outdir $outdir --img_number $img_number

##### 2. create dataset .yaml file (like coco_external)

Yolov5 algorithm parses yaml information: path to dataset(train, validation and test) in YOLO (darknet) format (see below); number of classes and classes names

In [3]:
#YAML Paths
datasetyaml=os.path.join(scriptdir, 'coco128_external.yaml')

##### 3. Select a Model(yolov5s, yolov5x, ...) and update the number of classes to match the dataset

Go to $./models$ folder and copy model .yaml file tho this directory and change the $nc: $ parameter

In [42]:
#YAML Paths
modelyaml=os.path.join(scriptdir, 'yolov5s.yaml')

## 1. Train
Run the training command below to train for X epochs. 

EXAMPLE: You can train YOLOv5s from scratch by passing `--cfg yolov5s.yaml --weights ''`, or train from a pretrained checkpoint by passing a matching weights file: `--cfg yolov5s.yaml --weights yolov5s.pt`.

Train function retrains the model with the new 'train' data and tests the model against the 'val' data

All training results are saved to runs/exp0 for the first experiment, then runs/exp1, runs/exp2 etc. for subsequent experiments.

In [38]:
# Clear any logs from previous runs
#BUG WINDOWS runs/exp0 don't iterates to exp1,exp2,...
logdir=os.path.join(yolov5_dir, 'runs/exp0')
!rm -rfv $logdir

In [39]:
# Start tensorboard (optional)
#BUG WINDOWS are not able to log information
%cd $yolov5_dir

%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir runs

C:\Users\nmc_costa\google_drive\projects\bosch_P19\research\python_ws\violent_action\yolo\yolov5


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 16248.

In [43]:
# Train Models from scractch 
#(#WARNING: if using --cache parameter you need to be careful: in yolov5.utils.datasets gives keyerror; #SOLUTION delete cache in dataset folder or don't use --cache)
%cd $yolov5_dir
#!pwd
# !ls
!python train.py --img 640 --batch 8 --epochs 3 --data $datasetyaml --cfg $modelyaml --weights ''

C:\Users\nmc_costa\google_drive\projects\bosch_P19\research\python_ws\violent_action\yolo\yolov5
Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
Namespace(batch_size=8, bucket='', cache_images=False, cfg='./models/yolov5s.yaml', data='C:\\Users\\nmc_costa\\google_drive\\projects\\bosch_P19\\research\\python_ws\\violent_action\\yolo\\mola8_external.yaml', device='', epochs=3, evolve=False, hyp='', img_size=[640], multi_scale=False, name='', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights="''")
Using CUDA device0 _CudaDeviceProperties(name='GeForce GTX 1070', total_memory=8192MB)

Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/
Hyperparameters {'optimizer': 'SGD', 'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, '


Scanning images: 100%|##########| 8/8 [00:00<00:00, 800.00it/s]
Traceback (most recent call last):
  File "train.py", line 404, in <module>
    train(hyp)
  File "train.py", line 164, in train
    dataloader, dataset = create_dataloader(train_path, imgsz, batch_size, gs, opt,
  File "C:\Users\nmc_costa\google_drive\projects\bosch_P19\research\python_ws\violent_action\yolo\yolov5\utils\datasets.py", line 50, in create_dataloader
    dataset = LoadImagesAndLabels(path, imgsz, batch_size,
  File "C:\Users\nmc_costa\google_drive\projects\bosch_P19\research\python_ws\violent_action\yolo\yolov5\utils\datasets.py", line 335, in __init__
    cache = self.cache_labels(cache_path)  # cache
  File "C:\Users\nmc_costa\google_drive\projects\bosch_P19\research\python_ws\violent_action\yolo\yolov5\utils\datasets.py", line 455, in cache_labels
    torch.save(x, path)  # save for next time
  File "C:\Tools\miniconda3\envs\violent_action\lib\site-packages\torch\serialization.py", line 369, in save
    


 12           [-1, 6]  1         0  models.common.Concat                    [1]                           
 13                -1  1    378624  models.common.BottleneckCSP             [512, 256, 1, False]          
 14                -1  1     33024  models.common.Conv                      [256, 128, 1, 1]              
 15                -1  1         0  torch.nn.modules.upsampling.Upsample    [None, 2, 'nearest']          
 16           [-1, 4]  1         0  models.common.Concat                    [1]                           
 17                -1  1     95104  models.common.BottleneckCSP             [256, 128, 1, False]          
 18                -1  1      2709  torch.nn.modules.conv.Conv2d            [128, 21, 1, 1]               
 19                -2  1    147712  models.common.Conv                      [128, 128, 3, 2]              
 20          [-1, 14]  1         0  models.common.Concat                    [1]                           
 21                -1  1    313088  

## 2. Output data (Visualize)

View runs/exp0/train*.jpg images to see training images, labels and augmentation effects. A Mosaic Dataloader is used for training (shown below), a new concept developed by Ultralytics and first featured in YOLOv4. If your labels are not correct in these images then you have incorrectly labelled your data, and should revisit 2. Create Labels.

In [ ]:
Image(filename=os.path.join(yolov5_dir,'runs/exp0/train_batch0.jpg'), width=900)

## 3. Inference (detect)

In [ ]:
%cd $yolov5_dir
!python detect.py --source D:/external_datasets/mola_samples/VIDEOS --output  D:/external_datasets/mola_samples/inference/yolov5/yolov5s_coco128test_augment --weights runs/exp0/weights/best.pt --conf 0.4 --augment

## 4. (Re-) Test/validate

(Re-) Test a model on dataset to determine trained accuracy (WARNING : Train already does this). Models are auto-downloaded from Google Drive. To show results by class use the --verbose flag.

In [ ]:
# SHELL Ipython: Run YOLOv5x on COCO validation images
%cd $yolov5_dir
#!pwd
#!ls
!python test.py --weights runs/exp0/weights/best.pt --data $yaml --task 'val' --img 672 --verbose